In [1]:
from typing import NamedTuple, List
import numpy as np
import pandas as pd

# Snake and ladder

- MDP representation :
    - States  : (1, n**2)
    - Actions : 
        (
            left_1, right_1, left_2, right_2 
            up, down, 
            [up_ladder], [down_snake], [terminate]
        )
    - Transition : 
        T(s, a, s')  = p=0.7
        T(s, a', s') = (1-p)=0.3  // a' is the reverse of a
    - Reward :
        R(s, a, s') = 1
    - Start state :
        (0, 0)
    - End state :
        (n-1, n-1)

- load and represent the board

___

In [2]:
class Actions:
    _actions = ["up", "right_1", "right_2", "left_1", "left_2", "down", "ladder_up", "snake_down", "terminate"]
    up, right_1, right_2, left_1, left_2, down, ladder_up, snake_down, terminate = _actions

    @classmethod
    def get_index(cls, action):
        return cls._actions.index(action)

    @classmethod
    def index_to_action(cls, index:int) :
        return cls._actions[index]
    
    @classmethod
    def get_reward(cls, action) :
        if action in ["up", "right_1", "left_1", "down"] :
            return -1
        elif action in ["left_2", "right_2"]:
            return -2
        elif action in ["terminate"] :
            return 100
        elif action in ["ladder_up"] : 
            return 10
        elif action in ["snake_down"] :
            return -10
            
    @classmethod    
    def get_num_actions(cls):
        return len(cls._actions)

In [3]:
def inbound(loc, x=0, y=0) :
    new_loc = (loc[0]+y, loc[1]+x)
    if (0 <= new_loc[0] < n) and (0 <= new_loc[1] < n) :
        return True
    return False

In [4]:
def get_actions(loc) -> List[Actions]:
    if loc_to_state(loc) == n**2 : # final state
        return [Actions.terminate]
    elif list(loc) in snakes_loc[:, -2:].tolist(): # if we are at the head of a snake
        return [Actions.snake_down]
    elif list(loc) in ladders_loc[:, 0:2].tolist(): # if we are at the bottom of a ladder
        return [Actions.ladder_up]
    else :
        actions = [Actions.right_1, Actions.right_2, Actions.left_1, Actions.left_2,]
        if (loc[0]%2 == 0) and (loc[1] == (n-1)) : # even rows
            actions.extend([Actions.up]) #  [..,Actions.down] if we could come down from sides
        elif (loc[0]%2 != 0) and (loc[1] == (0)) : # odd rows
            actions.extend([Actions.up]) #  [..,Actions.down] if we could come down from sides
            
        return actions
        # ? we also could have improved this by omitting the actions that are going to go out of the bounds of board
        # actions=[]
        # # 1 right
        # if inbound(loc, x=1) :
        #     actions.append(Actions.right_1)
        # elif inbound(loc, x=2) :


In [5]:
def loc_to_state(loc) -> int:
    '''Converts the (x,y) points to the state number of the actual game board
    loc = (x, y)
        x is even  --> (x*n) + (y+1)
        x is odd --> (x*n) + (n-y)
    '''
    x, y = loc
    if x%2 == 0 :
        return (x*n) + (y+1)
    if x%2 != 0: 
        return (x*n) + (n-y)
    

In [6]:
def state_to_loc(state) :
    '''
    we had these formulas for `loc_to_state` function, now by having x, we can 
    have y from the formula :
        x is even  --> (x*n) + (y+1) = state
        x is odd --> (x*n) + (n-y) = state
    '''
    x = (state - 1) // n
    # even row (starting from 0)
    if  x%2 == 0 :
        x = (state - 1) // n
        y = (state - (x*n)) - 1

    # odd row
    else :
        x = (state - 1) // n
        y = -1*((state - (x*n))-n)



    return (x, y)

In [7]:
def bellman(loc, new_loc, action) :
    if new_loc == -1 : # we are in the final step # * we can make this part better later...
        value = Actions.get_reward(action) # ? does it need Lambda to be multiplied to it 
    else :
        state = loc_to_state(loc)
        new_state = loc_to_state(new_loc)
        sample = Actions.get_reward(action) + lambda_*max(q_table[new_state-1, :])         # sample = R(s,a,s') + λ*max Q(s',a')
        value = (1 - alpha) * q_table[state-1, Actions.get_index(action)] + alpha*sample   # Q(s,a) = (1-a)*Q(s,a) + a*sample 
        
    return np.round(value, 3)

In [8]:
def move(loc, action) -> tuple|int:
    '''
    by receiving a loc and action, it will return the new location
    if the return value is -1 the program should terminate
    '''
    if action == Actions.right_1 :
        if inbound(loc, x=1) :
            new_loc = (loc[0], loc[1]+1)
        else :
            new_loc = loc

    elif action == Actions.right_2 :
        if inbound(loc, x=2) :
            new_loc = (loc[0], loc[1]+2)
        else :
            if inbound(loc, x=1) :
                new_loc = (loc[0], loc[1]+1)
            else :
                new_loc = loc

    elif action == Actions.left_1 :
        if inbound(loc, x=-1):
            new_loc = (loc[0], loc[1]-1)
        else :
            new_loc = loc

    elif action == Actions.left_2 :
        if inbound(loc, x=-2):
            new_loc = (loc[0], loc[1]-2)
        else :
            if inbound(loc, x=-1) :
                new_loc = (loc[0], loc[1]-1)
            else :
                new_loc = loc

    elif action == Actions.up :
        if inbound(loc, y=1):
            new_loc = (loc[0]+1, loc[1])
        else : # ? this case should not happen logically 🤔
            new_loc = loc

    elif action == Actions.down :
        if inbound(loc, y=-1):
            new_loc = (loc[0]-1, loc[1])
        else :
            new_loc = loc

    elif action == Actions.ladder_up :
        # we are currently at the bottom of the ladder, so we get it's top :
        current_state = loc_to_state(loc)
        for bottom, top  in ladders :
            if bottom == current_state :
                new_loc = state_to_loc(top)
                break

    elif action == Actions.snake_down :
        current_state = loc_to_state(loc)
        for bottom, top  in snakes :
            if top == current_state :
                new_loc = state_to_loc(bottom)
                break

    elif action == Actions.terminate :
        return -1

    else :
        raise Exception(f"action` {action} ` not found!")
    return new_loc

    

In [9]:
# def get_best_move(loc, actions) :
#     action_value = {}
#     for action in actions :
#         new_loc = move(loc, action)
#         value = bellman(loc, new_loc, action)
#         action_value[action] = value
    
#     # print(sorted(action_value, key=lambda k: action_value[k])[-1])
#     action = sorted(action_value, key=lambda k: action_value[k])[-1] # sort the dictionary based on the values and return the key and value
#     return (action, action_value[action])


In [10]:
def get_best_move(loc, actions) :
    state = loc_to_state(loc)
    if actions[0] == Actions.ladder_up :
        best_value = q_table[state-1, Actions.get_index(Actions.ladder_up)]
        best_action = Actions.ladder_up # which is also the only action
    elif actions[0] == Actions.snake_down :
        best_value = q_table[state-1, Actions.get_index(Actions.snake_down)]
        best_action = Actions.snake_down # which is also the only action

    if ((q_table[state-1, :]) == 0).all() : # when all the q-values have the same value, we choose one randomly
        best_value = 0
        best_action = np.random.choice(actions)
    else :
        action_indexes = [Actions.get_index(a) for a in actions]
        best_value = max(q_table[state-1, action_indexes]) 
        best_action_index = np.argmax(q_table[state-1, action_indexes])
        best_action = actions[best_action_index] # ! DONT DO THIS  `Actions.index_to_action(best_action_index)` it returns a complete wrong
                                                 # ! wrong answer. the reason is we change the indexing area by `action_indexes` in 
                                                 # ! `q_table[state-1, action_indexes]`

    return best_action, best_value


In [11]:
def update_value(q_table:np.array, loc:tuple, action:Actions, value:int) -> None:
    state = loc_to_state(loc)
    q_table[state-1, Actions.get_index(action)] = value

In [12]:
def _show_table(n: int) :
    board = np.arange((n**2), 0, step=-1).reshape(n, n)
    # reverse the odd rows
    if n%2 == 0 :
        odd_rows = np.apply_along_axis(
            func1d=(lambda r: r[::-1]),
            axis=1,
            arr=board[1::2, ]
        )
        board[1::2,] = odd_rows

    else :
        even_rows = np.apply_along_axis(
            func1d=(lambda r: r[::-1]),
            axis=1,
            arr=board[0::2, ]
        )
        board[0::2,] = even_rows

    return board

___

In [13]:
import time

In [23]:
n = 4
alpha = 0.3
p = 0.7
lambda_ = 1

snakes = [(4, 14)]
ladders = [(2, 9), (3, 12)]
snakes_loc = []
ladders_loc = []

# fix snake loc
for s in snakes :
    x_start, y_start = state_to_loc(s[0])
    x_end, y_end = state_to_loc(s[1])
    snakes_loc.append(
        ((x_start, y_start, x_end, y_end))
    )
# fix ladder loc
for l in ladders :
    x_start, y_start = state_to_loc(l[0])
    x_end, y_end = state_to_loc(l[1])
    ladders_loc.append(
        ((x_start, y_start, x_end, y_end))
    )

# * (x, y, x', y') -> (x, y) for start / (x', y') for the end

snakes_loc = np.array(snakes_loc)
ladders_loc = np.array(ladders_loc)

In [24]:
ladders_loc

array([[0, 1, 2, 0],
       [0, 2, 2, 3]])

In [25]:
q_table = np.zeros(shape=(n**2, Actions.get_num_actions()))

start_loc = (0, 0)
loc = start_loc

for _ in range(50) :
    actions = get_actions(loc)
    # if actions[0] == Actions.terminate :
    #     loc = start_loc
    #     actions = get_actions(loc)
    #     # print(max(q_table[loc_to_state(loc)-1, :]))
    #     # break
    print(loc)

    best_action, value = get_best_move(loc, actions)

    # with p=0.7 the best action will happen and with p=0.3 the reverse of it
    if best_action == Actions.up:  action = np.random.choice([Actions.up, Actions.down], p=[p, 1-p])
    elif best_action == Actions.right_1: action = np.random.choice([Actions.right_1, Actions.left_1], p=[p, 1-p])
    elif best_action == Actions.right_2: action = np.random.choice([Actions.right_2, Actions.left_2], p=[p, 1-p])
    elif best_action == Actions.left_1: action = np.random.choice([Actions.left_1, Actions.right_1], p=[p, 1-p])
    elif best_action == Actions.left_2: action = np.random.choice([Actions.left_2, Actions.right_2], p=[p, 1-p])
    elif best_action == Actions.down: action = np.random.choice([Actions.down, Actions.up], p=[p, 1-p])
    elif best_action == Actions.ladder_up: action = Actions.ladder_up
    elif best_action == Actions.snake_down: action = Actions.snake_down
    elif best_action == Actions.terminate: action = Actions.terminate

    new_loc = move(loc, action)
    if new_loc == -1 : # means we have reached the terminal
        new_value = bellman(loc, new_loc, action)
        update_value(q_table, loc, action, new_value)
        new_loc = (0, 0)
        
    else :
        new_value = bellman(loc, new_loc, action) # new value for the current q-value
        update_value(q_table, loc, action, new_value) # FIXME it doesn't update , check the log of this block output

    print(best_action, action)
    print(new_value)
    print(q_table[loc_to_state(loc)-1, :], end="\n\n")
    loc = new_loc
    # time.sleep(0.5)

(0, 0)
left_2 left_2
-0.6
[ 0.   0.   0.   0.  -0.6  0.   0.   0.   0. ]

(0, 0)
right_1 left_1
-0.3
[ 0.   0.   0.  -0.3 -0.6  0.   0.   0.   0. ]

(0, 0)
right_1 right_1
-0.3
[ 0.  -0.3  0.  -0.3 -0.6  0.   0.   0.   0. ]

(0, 1)
ladder_up ladder_up
3.0
[0. 0. 0. 0. 0. 0. 3. 0. 0.]

(2, 0)
right_1 right_1
-0.3
[ 0.  -0.3  0.   0.   0.   0.   0.   0.   0. ]

(2, 1)
right_1 right_1
-0.3
[ 0.  -0.3  0.   0.   0.   0.   0.   0.   0. ]

(2, 2)
left_2 left_2
-0.6
[ 0.   0.   0.   0.  -0.6  0.   0.   0.   0. ]

(2, 0)
right_2 right_2
-0.6
[ 0.  -0.3 -0.6  0.   0.   0.   0.   0.   0. ]

(2, 2)
right_1 right_1
-0.3
[ 0.  -0.3  0.   0.  -0.6  0.   0.   0.   0. ]

(2, 3)
left_1 left_1
-0.3
[ 0.   0.   0.  -0.3  0.   0.   0.   0.   0. ]

(2, 2)
right_2 right_2
-0.6
[ 0.  -0.3 -0.6  0.  -0.6  0.   0.   0.   0. ]

(2, 3)
right_1 left_1
-0.51
[ 0.    0.    0.   -0.51  0.    0.    0.    0.    0.  ]

(2, 2)
left_1 right_1
-0.51
[ 0.   -0.51 -0.6   0.   -0.6   0.    0.    0.    0.  ]

(2, 3)
right_1 l

In [26]:
_show_table(n)

array([[16, 15, 14, 13],
       [ 9, 10, 11, 12],
       [ 8,  7,  6,  5],
       [ 1,  2,  3,  4]])

In [27]:
# q_table

In [28]:
q_table_frame = pd.DataFrame(
    q_table,
    columns=Actions._actions,
    index=np.arange(1, (n**2)+1)
)
for row in q_table_frame.iterrows() :
    actions = get_actions(state_to_loc(row[0]))
    nan_actions = q_table_frame.loc[row[0], :].drop(actions).index
    q_table_frame.loc[row[0], nan_actions] = np.nan
    # print(actions)
q_table_frame

,up,right_1,right_2,left_1,left_2,down,ladder_up,snake_down,terminate
1,NaN,-0.300,-0.60,-0.300,-0.6,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,5.1,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,0.00,0.000,0.00,0.000,0.3,NaN,NaN,NaN,NaN
5,NaN,0.000,0.00,0.000,0.0,NaN,NaN,NaN,NaN
6,NaN,0.000,0.00,0.000,0.0,NaN,NaN,NaN,NaN
7,NaN,0.000,0.00,0.000,0.0,NaN,NaN,NaN,NaN
8,0.00,0.000,0.00,0.000,0.0,NaN,NaN,NaN,NaN
9,NaN,-0.657,-1.02,-0.657,-0.6,NaN,NaN,NaN,NaN
10,NaN,-0.657,-0.60,-0.510,-0.6,NaN,NaN,NaN,NaN


In [29]:
# optimal policy
q_table_frame.idxmax(axis=1)

1        right_1
2      ladder_up
3      ladder_up
4         left_2
5        right_1
6        right_1
7        right_1
8             up
9         left_2
10        left_1
11       right_2
12            up
13       right_1
14    snake_down
15       right_1
16     terminate
dtype: object

In [30]:
_show_table(n)

array([[16, 15, 14, 13],
       [ 9, 10, 11, 12],
       [ 8,  7,  6,  5],
       [ 1,  2,  3,  4]])

In [31]:
print(ladders, end="\n\n")
print(snakes)

[(2, 9), (3, 12)]

[(4, 14)]
